In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from functools import partial

import numpy as np
import pandas as pd
import torch as T
import torch.optim as optim

In [3]:
cd ..

C:\Projects\python\recommender


In [4]:
from datasets import TorchMovielen10k, TorchTopcoder, DataBunch
from models import FMLearner, TorchFM, TorchHrmFM, TorchPrmeFM, TorchTransFM
from models.fm_learner import simple_loss, trans_loss

### Datasets Settings

In [5]:
DEVICE = T.cuda.current_device()
BATCH = 1200
SHUFFLE = True
WORKERS = 0

In [6]:
movie_db = TorchMovielen10k(data_path="./inputs/ml-100k/u.data")

2019-09-12 11:01:22,911 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in ./inputs/ml-100k/u.data
2019-09-12 11:01:22,911 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in ./inputs/ml-100k/u.data
2019-09-12 11:01:22,911 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in ./inputs/ml-100k/u.data
I0912 11:01:22.911924  1112 torch_movielen.py:58] Read dataset in ./inputs/ml-100k/u.data
2019-09-12 11:01:22,927 - C:\Projects\python\recommender\utils.py - INFO - Original user size: 943
2019-09-12 11:01:22,927 - C:\Projects\python\recommender\utils.py - INFO - Original user size: 943
2019-09-12 11:01:22,927 - C:\Projects\python\recommender\utils.py - INFO - Original user size: 943
I0912 11:01:22.927881  1112 torch_movielen.py:62] Original user size: 943
2019-09-12 11:01:22,933 - C:\Projects\python\recommender\utils.py - INFO - Original item size: 1682
2019-09-12 11:01:22,933 - C:\Projects\python\recommender\utils.py - INFO - Original item size:

In [7]:
movie_db.config_db(batch_size=BATCH,
                   shuffle=SHUFFLE,
                   num_workers=WORKERS,
                   device=DEVICE)

### Config model

In [8]:
feat_dim = movie_db.feat_dim
NUM_DIM = 124
INIT_MEAN = 1

In [9]:
fm_model = TorchFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)

### Create criterion

In [10]:
# regst setting
LINEAR_REG = 1
EMB_REG = 1

In [11]:
simple_loss_callback = partial(simple_loss, LINEAR_REG, EMB_REG)
simple_loss_callback

functools.partial(<function simple_loss at 0x000001AA6A850378>, 1, 1)

### Create optimizer

In [12]:
LEARNING_RATE = 0.001
DECAY_FREQ = 1000
DECAY_GAMMA = 1

In [13]:
adam_opt = optim.Adam(fm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt, step_size=1000, gamma=DECAY_GAMMA)

In [14]:
fm_learner = FMLearner(fm_model, adam_opt, schedular, movie_db)
fm_learner

In [16]:
fm_learner.fit(epoch=1, loss_callback=simple_loss_callback)

  0%|                                                                                                                                                                                          | 0/1 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 173676.9300677166
Epoch 0 step 1: training loss: 173692.0360299736
Epoch 0 step 2: training loss: 172970.4643560509
Epoch 0 step 3: training loss: 171930.73986165793
Epoch 0 step 4: training loss: 171626.50870173992
Epoch 0 step 5: training loss: 171400.6234150797
Epoch 0 step 6: training loss: 170829.69956044471
Epoch 0 step 7: training loss: 169919.6318466643
Epoch 0 step 8: training loss: 170182.89578743622
Epoch 0 step 9: training loss: 169783.82539182692
Epoch 0 step 10: training loss: 169463.91613374345
Epoch 0 step 11: training loss: 169631.98646157497
Epoch 0 step 12: training loss: 168254.66129824377
Epoch 0 step 13: training loss: 168834.73228242598
Epoch 0 step 14: training loss: 168272.73576621377
Epoch 0 step 15: training loss: 167564.71810885606
Epoch 0 step 16: training loss: 167653.3140840673
Epoch 0 step 17: training loss: 167371.21773820912
Epoch 0 step 18: training loss: 166551.52635611556
Epoch 0 step 19: training loss: 166767

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:19<00:00, 19.39s/it]
